In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/audio1/test (2).wav


In [3]:
# Get the path of the audio file
import glob
audio_files = glob.glob('https://drive.google.com/file/d/1unZlfgSWa2Y2vzlJ-cwcIDYiPvJSo1SY/view?usp=drive_link')  # Adjust the file extension if necessary
path = audio_files

In [4]:
num_speakers = 2 #@param {type:"integer"}

language = 'any' #@param ['any', 'English']

model_size = 'large' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'

In [5]:
%pip install wheel setuptools pip --upgrade
%pip install wheel setuptools pip --upgrade
%python -m pip install wheel setuptools pip --upgrade
%python3 -m pip install wheel setuptools pip --upgrade
%py -m pip install wheel setuptools pip --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 12.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-api 1.17.0 requires importlib-metadata~=6.0.0, but you have importlib-metadata 5.2.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.10.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [6]:
%pip install numpy --use-deprecated=legacy-resolver
%pip install numpy --use-deprecated=legacy-resolver

%python -m pip install numpy --use-deprecated=legacy-resolver
%python3 -m pip install numpy --use-deprecated=legacy-resolver
%py -m pip install numpy --use-deprecated=legacy-resolver

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [7]:
%pip install -q git+https://github.com/openai/whisper.git > /dev/null
%pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [9]:
model = whisper.load_model(model_size)

100%|██████████████████████████████████████| 2.87G/2.87G [00:22<00:00, 134MiB/s]


In [10]:
 path = '/kaggle/input/audio1/test (2).wav'

In [11]:
result = model.transcribe(path)
segments = result["segments"]


In [13]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [14]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [15]:
try:
  embeddings = np.zeros(shape=(len(segments), 192))
  for i, segment in enumerate(segments):
    embeddings[i] = segment_embedding(segment)

  embeddings = np.nan_to_num(embeddings)

except AssertionError:
  print("assertion error")

assertion error


In [16]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [19]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("/kaggle/working/transcript.txt", "w")
""
for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [18]:
pwd

'/kaggle/working'

In [22]:
myfile=open('transcript.txt')

In [23]:
myfile.read()

'\nSPEAKER 1 0:00:00\nआपको किसे बेजाया है? प्राधान वो ही तो बता रहा हूं आप समझने को तयीर नहीं है कॉंस्टिटूषन में प्राधान है? यह मानिताओं का उत्तर है? आप फ्रीज प्राधान वो जो प्राधान है जो बता रहा हूं आप तो प्राभ़ान बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है प्राधान वो जो बता रहे है 